## Imports

In [1]:
import nltk
from sklearn.feature_extraction import text
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import torch
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import torch
import numpy as np


nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


## Load Datasets

In [22]:
# Load your data
data = pd.read_json('../data/data.jsonl', lines=True)
test_data = pd.read_json('../data/test_final.jsonl', lines=True)
train_data = pd.read_json('../data/train_final.jsonl', lines=True)
validation_data = pd.read_json('../data/validation_final.jsonl', lines=True)

# Remove duplicates
test_data = test_data.drop_duplicates(subset=['text'])
train_data = train_data.drop_duplicates(subset=['text'])
validation_data = validation_data.drop_duplicates(subset=['text'])


## Pre-processing

In [4]:
my_stop_words = text.ENGLISH_STOP_WORDS
words_to_keep = frozenset(['no', 'couldnt', 'cry', 'not', 'cant', 'cannot', 'nor', 'except', 'nobody',
                           'off', 'but', 'serious', 'enough', 'nothing', 'alone', 'down', 'only', 'without'])
my_stop_words = my_stop_words - words_to_keep

def pre_process_data(dataset):
    # Tokenize
    dataset['text'] = dataset['text'].apply(word_tokenize)
    # Remove stop words
    dataset['text'] = dataset['text'].apply(lambda x: [word for word in x if word.lower() not in my_stop_words])
    return dataset


train_data = pre_process_data(train_data)
validation_data = pre_process_data(validation_data)
test_data = pre_process_data(test_data)


['woke', 'feeling', 'particularly', 'vile', 'tried', 'ignore', 'but', 'got', 'worse', 'worse', 'worse']
0       [feel, awful, s, job, position, succeed, just,...
1                                [im, alone, feel, awful]
2       [ive, probably, mentioned, but, really, feel, ...
3                             [feeling, little, low, day]
4       [beleive, sensitive, people, feeling, tend, co...
                              ...                        
5395              [feel, grumpy, haven, t, yoga, ed, day]
5396    [read, blog, suburb, direction, mentioned, cas...
5397    [not, feel, thing, realize, violent, physical,...
5398    [feel, petty, silly, giving, shit, but, little...
5399    [remember, feeling, bitter, couldnt, pop, ball...
Name: text, Length: 5396, dtype: object


## Tokenization Using BERT Tokenizer:
BERT tokenizer requires the data to be in a specific format

???????? MAS ESTÁ CERTO ANTES TAMBEM TOKENIZARMOS ????????


* podemos usar o bert-base-uncased e o roberta-based e comparar dps
* maybe usar embeddings tb depois


In [23]:
# Initialize the BERT tokenizer and model
model_name = "bert-base-uncased"

# SUPOSTAMENTE ESTE TB É BOM
# model_name = "roberta-base"


tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Tokenize using BERT tokenizer
def tokenize_data(texts):
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128)

train_encodings = tokenize_data(train_data['text'].apply(lambda x: ' '.join(x)).tolist())
val_encodings = tokenize_data(validation_data['text'].apply(lambda x: ' '.join(x)).tolist())
test_encodings = tokenize_data(test_data['text'].apply(lambda x: ' '.join(x)).tolist())

c:\Users\inesc\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Create Dataset Objects

In [24]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TextDataset(train_encodings, train_data['label'].tolist())
val_dataset = TextDataset(val_encodings, validation_data['label'].tolist())
test_dataset = TextDataset(test_encodings, test_data['label'].tolist())

## Training

In [30]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    preds = np.argmax(predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted')
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


#### Training hyppertunning

In [43]:
## depois coloco aqui quando passar o trainer parar de me dar erro ?????

In [39]:
# # Training arguments
# training_args = TrainingArguments(
#     output_dir='./results',
#     num_train_epochs=3,
##    per_device_train_batch_size=8,
##     per_device_eval_batch_size=8,
#     evaluation_strategy="epoch",
#     logging_dir='./logs',
# )

# # Initialize Trainer
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=val_dataset,
#     compute_metrics=compute_metrics
# )



## Train and Evaluate

In [40]:
# trainer.train()
# eval_results = trainer.evaluate()
# print(eval_results)

## Predictions

In [41]:
# predictions = trainer.predict(test_dataset)
# preds = np.argmax(predictions.predictions, axis=1)
# print(preds)
